In [1]:
from alphatrade import *

# import logging
# logging.basicConfig(level=logging.DEBUG)

from alphatrade import AlphaTrade
import config
import pyotp
import pandas as pd
import time
from datetime import datetime
from datetime import timedelta
import sqlalchemy




In [2]:
Totp = config.Totp
pin = pyotp.TOTP(Totp).now()
totp = f"{int(pin):06d}" if len(pin) <=5 else pin   
sas = AlphaTrade(login_id=config.login_id, password=config.password, twofa=totp, access_token=config.access_token)

In [ ]:
import pandas as pd
import time
import math
import pytz
from IPython.display import clear_output


previous_net_quantity = None
previous_trading_symbol = None

while True:
    positions = sas.get_daywise_positions()['data']['positions']
    positions_df = pd.DataFrame(positions)
    positions_df = positions_df[['net_quantity', 'trading_symbol', 'realised_pnl', 'instrument_token', 'ltp']]

    for _, row in positions_df.iterrows():
        net_quantity = row['net_quantity']
        trading_symbol = row['trading_symbol']
        instrument_token = row['instrument_token']

        if net_quantity > 0 and (net_quantity != previous_net_quantity or trading_symbol != previous_trading_symbol):
            clear_output(wait=False)
            tbook = sas.get_trade_book()
            df_tbook = pd.DataFrame(tbook['data']['trades'])
            # Convert Unix timestamp to datetime format
            df_tbook['exchange_time'] = pd.to_datetime(df_tbook['exchange_time'], unit='s')
            # Convert datetime to desired timezone (Asia/Kolkata)
            timezone = pytz.timezone('Asia/Kolkata')
            df_tbook['exchange_time'] = df_tbook['exchange_time'].dt.tz_localize(pytz.utc).dt.tz_convert(timezone)
            df_tbook['oms_order_id'] = df_tbook['oms_order_id'].astype('int64')
            df_tbook['exchange_order_id'] = df_tbook['exchange_order_id'].astype('int64')

            # Convert datetime to desired time format (hh:mm:ss)
            df_tbook['exchange_time'] = df_tbook['exchange_time'].dt.strftime('%H:%M:%S')
            df_tbook.to_csv('tbook2.csv', index=False)  # Save DataFrame to CSV file without the index

            trade_book_df = pd.read_csv('tbook2.csv')  # Load CSV file into DataFrame

            filtered_trades = trade_book_df[(trade_book_df['trading_symbol'] == trading_symbol) & (trade_book_df['transaction_type'] == 'BUY')]

            cumulative_quantity = 0
            weighted_sum = 0

            for _, trade_row in filtered_trades.iterrows():
                trade_quantity = trade_row['filled_quantity']
                trade_price = trade_row['trade_price']

                # Check if adding the trade quantity exceeds the net_quantity
                if cumulative_quantity + trade_quantity > net_quantity:
                    remaining_quantity = net_quantity - cumulative_quantity
                    weighted_sum += remaining_quantity * trade_price
                    cumulative_quantity += remaining_quantity
                    break

                weighted_sum += trade_quantity * trade_price
                cumulative_quantity += trade_quantity

            average_trade_price = weighted_sum / cumulative_quantity if cumulative_quantity > 0 else 0
            target_trade_price = math.floor((average_trade_price * 1.01) * 10) / 10
            sl_trade_price = math.floor((average_trade_price * 0.97) * 10) / 10
            be_trade_price = math.floor((average_trade_price * 1.0) * 10) / 10
            print(f"trading_symbol = {trading_symbol}, net_quantity = {net_quantity}, average_price = {average_trade_price},instrument_token = {instrument_token}")
            print(f"Target = {target_trade_price}, SL_price = {sl_trade_price}, break_even = {be_trade_price}")

            # Place sell order
            quantity = net_quantity
            price = target_trade_price
            sbl = trading_symbol

            print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%1%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
            print(
                sas.place_order(transaction_type=TransactionType.Sell,
                                instrument=sas.get_instrument_by_token('NFO', instrument_token),
                                quantity=quantity,
                                order_type=OrderType.Limit,
                                product_type=ProductType.Intraday,
                                price=price,
                                trigger_price=None,
                                stop_loss=None,
                                square_off=None,
                                trailing_sl=None,
                                is_amo=False)
            )

        previous_net_quantity = net_quantity
        previous_trading_symbol = trading_symbol
        time.sleep(0.5)


    time.sleep(0.55)


trading_symbol = NIFTY2362218800PE, net_quantity = 200, average_price = 61.5,instrument_token = 57676
Target = 62.1, SL_price = 59.6, break_even = 61.5
%%%%%%%%%%%%%%%%%%%%%%%%%%%%1%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
{'status': 'success', 'message': 'Order placed successfully', 'data': {'oms_order_id': '230619000223876'}}
